In [20]:
import pandas as pd
# Define a function to load data dynamically
def load_data(file_path):
    # Load data from each sheet into separate dataframes
    locations_df = pd.read_excel(file_path, sheet_name='locations', usecols="A").dropna()
    nurses_df = pd.read_excel(file_path, sheet_name='nurses', usecols="A:B").dropna()
    patients_df = pd.read_excel(file_path, sheet_name='patients', usecols="A:C").dropna()
    task_execution_time_df = pd.read_excel(file_path, sheet_name='task_execution_time', usecols="A:B").dropna()
    medication_adherence_df = pd.read_excel(file_path, sheet_name='medication_adherence', usecols="A:H").replace('Not Applicable', pd.NA).dropna(how='all', subset=['M', 'T', 'W', 'Th', 'F', 'S', 'Su'])
    physical_therapy_adherence_df = pd.read_excel(file_path, sheet_name='physical_therapy_adherence', usecols="A:H").replace('Not Applicable', pd.NA).dropna(how='all', subset=['M', 'T', 'W', 'Th', 'F', 'S', 'Su'])
    distance_matrix_df = pd.read_excel(file_path, sheet_name='distance_matrix', header=None).dropna(how='all')

    # Process data
    task_execution_time_df['Time'] = task_execution_time_df['Time'].str.extract('(\d+)').astype(int)
    nurses_df['skillset'] = nurses_df['skillset'].str.split(', ')
    patients_df['needs'] = patients_df['needs'].str.split(', ')

    for col in ['M', 'T', 'W', 'Th', 'F', 'S', 'Su']:
        medication_adherence_df[col] = pd.to_numeric(medication_adherence_df[col] * 100, errors='coerce')
        physical_therapy_adherence_df[col] = pd.to_numeric(physical_therapy_adherence_df[col] * 100, errors='coerce')

    for col in ['M', 'T', 'W', 'Th', 'F', 'S', 'Su']:
        medication_adherence_df[col] = (medication_adherence_df[col]).astype('Int64', errors='ignore')
        physical_therapy_adherence_df[col] = (physical_therapy_adherence_df[col]).astype('Int64', errors='ignore')
    
    
    return {
        "locations": locations_df,
        "nurses": nurses_df,
        "patients": patients_df,
        "task_execution_time": task_execution_time_df,
        "medication_adherence": medication_adherence_df,
        "physical_therapy_adherence": physical_therapy_adherence_df,
        "distance_matrix": distance_matrix_df
    }


# Load data dynamically
file_path = './nurse_schedule_data_small_VA (1).xlsx'
data = load_data(file_path)

# Extract individual dataframes from the data dictionary
locations_df = data["locations"]
nurses_df = data["nurses"]
patients_df = data["patients"]
task_execution_time_df = data["task_execution_time"]
medication_adherence_df = data["medication_adherence"]
physical_therapy_adherence_df = data["physical_therapy_adherence"]
distance_matrix_df = data["distance_matrix"]

# Display the first few rows of each dataframe to verify the dynamic data loading
for name, df in data.items():
    print(f"{name}:\n{df.head()}\n")

days = ['M', 'T', 'W', 'Th', 'F', 'S', 'Su']

locations:
     Unnamed: 0
0       Roanoke
1  Williamsburg
2       Norfolk
3    Chesapeake

nurses:
        id                                           skillset
0  Nurse_1  [medication, personal hygiene assistance, phys...
1  Nurse_2  [medication, administering injections, physica...
2  Nurse_3  [physical therapy, medication, administering i...
3  Nurse_4  [wound care, personal hygiene assistance, draw...
4  Nurse_5  [medication, personal hygiene assistance, draw...

patients:
          id                                   needs      location
0  Patient_1  [administering injections, medication]       Norfolk
1  Patient_2                            [wound care]       Roanoke
2  Patient_3                            [medication]  Williamsburg
3  Patient_4              [administering injections]  Williamsburg
4  Patient_5                            [wound care]       Norfolk

task_execution_time:
                       Task  Time
0                medication    37
1             drawing blo

In [35]:
from gurobipy import Model, GRB, quicksum
import numpy as np

model = Model("nurse_scheduling")

# Get the number of nurses, patients, tasks, and days
num_nurses = len(nurses_df)
num_patients = len(patients_df)
num_tasks = len(task_execution_time_df)
num_days = 7  # Days in a week

# Define Decision Variables
x = model.addVars(num_nurses, num_patients, num_days, vtype=GRB.BINARY, name="x")
w = model.addVars(num_nurses, num_days, vtype=GRB.BINARY, name="w")
l = model.addVars(num_nurses, len(locations_df), num_days, vtype=GRB.BINARY, name="l")
t = model.addVars(num_nurses, num_patients, num_tasks, num_days, vtype=GRB.BINARY, name="t")

# Define the Objective Function
# The objective function aims to maximize the total adherence to medication and physical therapy schedules across all patients and days. 
# It does this by summing up the adherence percentages (multiplied by 100 and converted to integers) for each patient on each day 
# (if that day is present in the respective adherence dataframes) and assigning the tasks to the nurses in a way that maximizes this total adherence.
model.setObjective((
    quicksum(x[i, j, k] * row[days[k]] for i in range(num_nurses) for j, row in medication_adherence_df.iterrows() for k in range(num_days) if days[k] in medication_adherence_df.columns) +
    quicksum(x[i, j, k] * row[days[k]] for i in range(num_nurses) for j, row in physical_therapy_adherence_df.iterrows() for k in range(num_days) if days[k] in physical_therapy_adherence_df.columns)
), GRB.MAXIMIZE)

model.optimize()

# After model.optimize()

if model.status == GRB.OPTIMAL:
    print('Objective Value: ', model.objVal)
    for v in model.getVars():
        if v.x != 0:
            print(f'{v.varName}: {v.x}')
else:
    print('No optimal solution found. Status code:', model.status)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 0 rows, 2625 columns and 0 nonzeros
Model fingerprint: 0xa200058f
Variable types: 0 continuous, 2625 integer (2625 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [2e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 10925.000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 10 available processors)

Solution count 1: 10925 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.092500000000e+04, best bound 1.092500000000e+04, gap 0.0000%
Objective Value:  10925.0
x[0,0,0]: 1.0
x[0,0,1]: 1.0
x[0,0,2]: 1.0
x[0,0,3]: 1.0
x[0,0,4]: 1.0
x[0,0,5]: 1.0
x[0,0,6]: 1.0
x[0,2,0]: 1.0
x[0,2,1]: 1.0
x[0,2,2]: 1.0
x[0,2,3]: 1.0
x[0,2,4]: 1.0
x[0,2,5]: 1.0